In [147]:
import itertools as itools
import re
import numpy as np
from mpl_toolkits.mplot3d import Axes3D  
import matplotlib.pyplot as plt


class Point3D:
    
    def __init__(self, vector = None):
        
        if vector is not None:
            self.coord = np.array(vector)
        else:
            self.coord = np.array([0, 0, 0])
            
        return
    
    
    def modulo(self):
        
        return sum(abs(coord) for coord in self.coord)
    
    
class MoonObject:
    
    def __init__(self, position = None, velocity = None):
        # position can be initialized by passing values when instantiating object
        self.position = Point3D(position)
        # velocities are instead always initialized as 0 in this first case
        self.velocity = Point3D(velocity)
        
        return


class NbodySystem:
    
    def __init__(self, bodies = None):
        
        if bodies is not None:
            self.bodies = bodies
        
        else:
            self.bodies = dict()
            
        return
    
    
    def integration_step(self):
            
        for pair in itools.combinations(list(self.bodies), 2):
            
            body1 = self.bodies[pair[0]]
            body2 = self.bodies[pair[1]]
            
            # Updating velocities
            for i in range(3):
                if body1.position.coord[i] > body2.position.coord[i]:
                    body1.velocity.coord[i] -= 1
                    body2.velocity.coord[i] += 1
                elif body1.position.coord[i] < body2.position.coord[i]:
                    body1.velocity.coord[i] += 1
                    body2.velocity.coord[i] -= 1
                    
        # Updating position AFTER the update of velocities
        for key in self.bodies:
            self.bodies[key].position.coord += self.bodies[key].velocity.coord
        

        return
    
    
    def total_energy(self):
        
        total = 0
        
        for key in self.bodies:
            potential = self.bodies[key].position.modulo()
            kinetic = self.bodies[key].velocity.modulo()
            total += potential*kinetic
            
        return total
    
    
    def print_system(self):
        
        for key in self.bodies:
            body = self.bodies[key]
            print(key, "<", body.position.coord," > --- <", body.velocity.coord, " >")
        
        print("Total energy: ", self.total_energy())
        print('\n')
        
        return

In [148]:
initial_positions = []
with open('input.txt', 'r') as infile:
    for line in infile:
        if line == '\n':
            continue
        else:
            found = re.findall('-?[0-9]*', line) # <--- Look at this bad boy, I'm so proud!
            
            initial_positions.append([int(x) for x in found if x is not ''])
            
print(initial_positions)

[[6, 10, 10], [-9, 3, 17], [9, -4, 14], [4, 14, 4]]


In [149]:
# Names of the moons
moon_names = ['Io', 'Europa', 'Ganymede', 'Callisto']

# Create dictionary to pass to NbodySystem __init__() function
moon_dict = dict()
for name, pos in zip(moon_names, initial_positions):
    moon_dict[name] = MoonObject(position = pos)
    
# Initialize system
moon_system = NbodySystem(moon_dict)
moon_system.print_system()

Io < [ 6 10 10]  > --- < [0 0 0]  >
Europa < [-9  3 17]  > --- < [0 0 0]  >
Ganymede < [ 9 -4 14]  > --- < [0 0 0]  >
Callisto < [ 4 14  4]  > --- < [0 0 0]  >
Total energy:  0




In [150]:
# Counter for timesteps
step_count = 0

# Variable to save the paths of the moons. Used for plotting purposes later
moons_path = np.zeros((4, 1000, 3))

# Integration cycle
while step_count <= 999:
    
    # Save current position of satellites
    for i, key in enumerate(moon_system.bodies):
        moons_path[i][step_count][:] = moon_system.bodies[key].position.coord
    
    # Integrate system for one step
    moon_system.integration_step()
    # Increase counter
    step_count += 1
    
# Print information for system in last configuration
moon_system.print_system()

Io < [ 57 -19  -6]  > --- < [ 2 -3 14]  >
Europa < [-13 -62  75]  > --- < [ 15 -18  -1]  >
Ganymede < [ 21 133 -19]  > --- < [ 0 19 -3]  >
Callisto < [-55 -29  -5]  > --- < [-17   2 -10]  >
Total energy:  13045




In [146]:
%matplotlib 
# Print trajectories for the first number of timesteps specified here. Set '-1' to print 
# whole trajectories
timesteps = 50

# Declare figure
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')

# Plot the 4 trajectories
for i, key in enumerate(moon_system.bodies):
    ax.plot(moons_path[i,0:timesteps,0], moons_path[i,0:timesteps,1], moons_path[i,0:timesteps,2], label = key)

plt.legend()
plt.show()

Using matplotlib backend: TkAgg
